## Wikipedia tool

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
wiki.name

'wikipedia'

## Custom tool

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load();

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A1FE67AD70>)

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!" )

In [11]:
retriever_tool.name

'langsmith_search'

## Arxiv tool

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
tools = [wiki, arxiv, retriever_tool]

In [16]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\WORKSPACE\\GitHub\\langchain-projects\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A195EA3E20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<l

### Agents

In [17]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [19]:
from langchain import hub
# Get the prompt to use
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [22]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [24]:
agent_executor.invoke({"input": "Explain about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the 

{'input': 'Explain about LangSmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring that you can ship quickly and with confidence. LangSmith works independently and does not require the use of LangChain. \n\nKey features and steps to get started with LangSmith include:\n1. Install LangSmith using Python and Typescript SDKs.\n2. Create an API key for authentication.\n3. Set up your environment by configuring necessary variables.\n4. Log your first trace using the provided methods.\n\nLangSmith supports various workflows at each stage of the LLM application lifecycle, including tracing, evaluation, pricing, self-hosting, proxy capabilities, and more. It also offers additional resources such as the LangSmith Cookbook, LangChain Python and TypeScript documentation, and a Discord community for discussions.\n\nIf you have sensitive data or require private deplo

In [25]:
agent_executor.invoke({"input":"Create a mindmap for body building"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Bodybuilding`


Page: Bodybuilding
Summary: Bodybuilding is the practice of progressive resistance exercise to build, control, and develop one's muscles via hypertrophy. An individual who engages in this activity is I have retrieved information about bodybuilding. Now, I will create a mindmap based on the key points from the summary.
I will create a mindmap for bodybuilding based on the information retrieved. Let's start by outlining the key points:

1. Definition of Bodybuilding
2. Practice of Progressive Resistance Exercise
3. Muscle Building and Hypertrophy
4. Benefits of Bodybuilding
5. Nutrition and Diet
6. Training Techniques
7. Competitions and Events

Let's create the mindmap with these key points.

> Finished chain.


{'input': 'Create a mindmap for body building',
 'output': "I have retrieved information about bodybuilding. Now, I will create a mindmap based on the key points from the summary.\nI will create a mindmap for bodybuilding based on the information retrieved. Let's start by outlining the key points:\n\n1. Definition of Bodybuilding\n2. Practice of Progressive Resistance Exercise\n3. Muscle Building and Hypertrophy\n4. Benefits of Bodybuilding\n5. Nutrition and Diet\n6. Training Techniques\n7. Competitions and Events\n\nLet's create the mindmap with these key points."}

In [26]:
agent_executor.invoke({"input":"What is the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': 'What is the paper 1605.08386 about?',
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}